In [1]:
import graphBrowser as gB

In [2]:
gB.labelDict

{}

In [3]:
construct = gB.createAirportQuery('BOS')
elements = gB.createGraph('BOS')

In [5]:
import json
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from demos import dash_reusable_components as drc

app = JupyterDash(__name__)

app.layout = html.Div([
     
    html.Div(className='a',children = [
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='circle',
        clearable=False,
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['grid', 'random', 'circle', 'cose', 'concentric']
        ]
    )]),
    drc.NamedRadioItems(
                    name='Select',
                    id='radio-option',
                    options=drc.DropdownOptionsList(
                        'Show Neighbors'
#                         ,
#                         'Hide Neighbors'
                    ),
                    value='Show Neighbors'
                ),

    html.Div(className='b',children = [
    cyto.Cytoscape(
        id='cytoscape-update-layout',
        layout={'name': 'circle'},
        style={'width': '100%', 'height': '70vh'},
        elements=elements,
        stylesheet = [
            {
            'selector': 'node',
            'style': {
                'content': 'data(label)',
                'background-color': '#58FAF4',
                }
            },
            {
            'selector': 'edge',
            'style': {
                'content': 'data(labelLabel)',
                'curve-style': 'bezier',
                'target-arrow-color': 'black',
                'target-arrow-shape': 'triangle',
                'line-color': 'black'
                }
            }
        ]
    )]),
    html.P(id='cytoscape-tapNodeData-output'),
    html.P(id='cytoscape-tapEdgeData-output')
])


##########CALLBACKS##########
@app.callback(Output('cytoscape-tapNodeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapNodeData')])
def displayTapNodeData(data):
#     print('HH')
    if data:
        return "Airport selected: " + data['id']


@app.callback(Output('cytoscape-tapEdgeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapEdgeData')]
             )
def displayTapEdgeData(data):
    if data:
        return "Connection between " + data['source'].upper() + " and " + data['target'].upper() + " with edge value: " + data['label']

    
@app.callback(Output('cytoscape-update-layout', 'layout'),
              [Input('dropdown-update-layout', 'value')])
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

#expand node by neighbors
@app.callback(Output('cytoscape-update-layout', 'elements'),
              [Input('cytoscape-update-layout', 'tapNodeData')],
              [State('cytoscape-update-layout', 'elements')],
              [State('radio-option', 'value')]
             )
def generate_elements(data,e,options):
#     print("abcd",data)
#     print("Radio Options",options)
    
    print("AA")
    if not data:
        return e
    
    print("Data",data['expanded'])
    nodeURI = data['id']
    if options == "Show Neighbors":
        
        if data['expanded'] == True:
            print("1")
            return e
    
#     print("Label",data)
    
        if data and e:
        
            #changing extended to True for the node
            for element in e:
                if data['id'] == element.get('data').get('id'):
                    element['data']['expanded'] = True
    #                 print("Curr Node", element)
                    break
                
            
    #         print('node',nodeURI)
    #         print("Label",label)
    #         print('Here', type(e))
            df = gB.getNeighbours(nodeURI)
            nodes = gB.generateNodes(df,nodeURI)
#             print("Neighbors",nodes)
            edges = gB.generateEdges(df)

            for node in nodes:
                if node not in e:
                    e.append(node)
                
            for edge in edges:
                if edge not in e:
                    e.append(edge)        
                
        return e
    elif options == "Hide Neighbors":
        
        if data['expanded'] == False:
            print("2")
            return e
        return gB.removeNodes(e,nodeURI)

            
            
port = '8098'
host = '127.0.0.1'

app.run_server(mode='inline',port = port, host = host)

# print('App running on http://'+ host +':' + port +'/')

AA
AA
Data False
AA
Data False
